In [3]:
import pandas as pd
import math
import numpy as np
import os
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, Legend, HoverTool, Label
from bokeh.transform import dodge
import bokeh.palettes
output_notebook()

Loading BokehJS ...

In [40]:
path = '/Users/tuethomsen28/Google Drev/SocialDataVizz/Data'

focus_municipality = ["Albertslund","Ballerup","Brøndby","Dragør","Frederiksberg","Gentofte","Gladsaxe",
                           "Glostrup","Herlev","Hvidovre","Høje-Taastrup","Ishøj","København",
                           "Rødovre","Tårnby","Vallensbæk"]

df_hast_daily = pd.read_csv(os.path.join(path,"speedPerMun.csv"),sep=";")
df_hast_daily = df_hast_daily.set_index("Date")

df_avg_hast_pre_lockdown = df_hast_daily[df_hast_daily.index < "2020-03-13"].mean(0)
df_avg_hast_post_lockdown = df_hast_daily[df_hast_daily.index >= "2020-03-13"].mean(0)

df_avg_hast = pd.DataFrame((df_avg_hast_post_lockdown - df_avg_hast_pre_lockdown)/df_avg_hast_pre_lockdown,
                             columns=["pct_increase"])

df_avg_hast = df_avg_hast.reset_index()
df_avg_hast.columns = ["Municipality","pct_increase"]
df_avg_hast = df_avg_hast.set_index("Municipality")

#Creating format fitting for bokeh
source = ColumnDataSource(df_avg_hast)

#Creating a list for the x-axis on the bar-chart. 
x_range = [str(h) for h in source.data['Municipality']]

#Generating 14 colors for the different focus crimes  
color = bokeh.palettes.Category20[20][2]

bar ={} # to store vbars

#Creating figure frame with title, x -and y labels and disabling toolbar. 
p = figure(x_range=FactorRange(factors=x_range),
           title='Increase in speed per municipality',
           x_axis_label='Municipality',
           y_axis_label='Increase in speed (%)',
           plot_width=600,plot_height=400)


r = p.vbar("Municipality",  top="pct_increase", source= source, width=0.8, 
             fill_alpha=1,line_color=None,
             color=color,visible=True)

p.xaxis.major_label_orientation = math.pi/4

mun = r.name #extracting the name of each rendered crime
hover = HoverTool(tooltips=[
    ('Increase in speed (%)' , '@pct_increase{%0.2f}') #@ - extracts the fractions from each focuscrime
], renderers=[r]) # assigns the hoved effect to the correct bar rendering

p.add_tools(hover)
    
p.y_range.start = 0 #setting the y range from zero

show(p)